In [26]:
# load in csvOut.csv data (holds image name & classification)
import csv
import os #to get all picture in directory
from PIL import Image #to open images
import numpy as np #load in numpy module
import sys

# loads in full csvOut data (name & classification)
with open('..//annotations_handheld.csv', newline='') as csvfile:
    cls_full = list(csv.reader(csvfile))

In [27]:
# Get the list of all files in directory
path = "..//images_handheld_resized"
dir_list = os.listdir(path)

# Initialize list of 'keys'
keys = ["rgb", "bgr", "brg", "gbr", "grb", "rbg"]

# Initialize lists to hold bitmap information for each image
bitmap = {}
for key in keys:
    bitmap[key] = []

# list of classifications
cls = []

# loop through files and get bit map for each (save as object where filename => bitmap for r,g,b)
for index, file in enumerate(cls_full):
    
    # stop at 100 files for time being
    if index == 100:
        break
    
    # push classification to list
    cls.append(file[1])
    
    # method found https://stackoverflow.com/questions/46385999/transform-an-image-to-a-bitmap
    img = Image.open(path + "\\" + file[0])
    
    # resize image
    #small_img = img.resize((600, 400))
    #smaller_img = img.resize((300, 200))
    smallest_img = img.resize((120, 80))
    
    # split into 3 channels
    r, g, b = smallest_img.split()
    
    # merge channels in new order
    bitmap['rgb'].append(np.array(Image.merge('RGB', (r, g, b))).reshape(-1))
    bitmap['bgr'].append(np.array(Image.merge('RGB', (b, g, r))).reshape(-1))
    bitmap['brg'].append(np.array(Image.merge('RGB', (b, r, g))).reshape(-1))
    bitmap['gbr'].append(np.array(Image.merge('RGB', (g, b, r))).reshape(-1))
    bitmap['grb'].append(np.array(Image.merge('RGB', (g, r, b))).reshape(-1))
    bitmap['rbg'].append(np.array(Image.merge('RGB', (r, b, g))).reshape(-1))
    


In [28]:
# convert bitmaps to indexed np array
bitmap_arr = {}
for key in keys:
    bitmap_arr[key] = np.array(bitmap[key])

# convert classifications to numpy array
cls_np = np.array(cls)

In [31]:
# Training model (load necessary libraries)
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import GridSearchCV

# Initialize dictionary to hold training & test data
X_train = {}
X_test = {}
y_train = {}
y_test = {}

# loop through keys split data into training and testing sets
for key in keys: 
    X_train[key], X_test[key], y_train[key], y_test[key] = train_test_split(bitmap_arr[key], cls_np, test_size=0.2)

In [33]:
# set dictionary to hold predictions
y_pred = {}

# loop through keys, create SVM model and get classification prediction
for key in keys: 
    model = svm.SVC()
    model.fit(X_train[key], y_train[key])
    y_pred[key] = model.predict(X_test[key])

In [35]:
# check if metamorphic relations hold
if np.array_equal(y_pred['rgb'], y_pred['brg']):
    print("Metamorphic relation holds.")
else:
    print("Metamorphic relation does not hold.")

Metamorphic relation does not hold.


In [36]:
y_pred['rgb']

array(['H', 'B', 'B', 'H', 'B', 'B', 'B', 'H', 'B', 'B', 'B', 'B', 'B',
       'B', 'H', 'B', 'B', 'B', 'B', 'B'], dtype='<U1')

In [37]:
y_pred['brg']

array(['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'H',
       'B', 'B', 'B', 'H', 'B', 'H', 'B'], dtype='<U1')